In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Projects\\Deep_learning\\Deep-learning-model'

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path




In [21]:
from Hand_gesture_clf.constants import *
from Hand_gesture_clf.utils.common import read_yaml, create_diorectories

In [25]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_diorectories([self.config.artifacts_root])


    def get_prepare_callbacksd_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_chkpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_diorectories([
            Path(model_chkpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config



In [14]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time

In [29]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp =time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbaks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbaks
        ]
    
    

In [30]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacksd_config()
    prepare_callback = PrepareCallbacks(config = prepare_callback_config)
    call_back_list = prepare_callback.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2024-10-03 00:30:49,703 : INFO: common: yaml file config\config.yaml loaded successfully]
[2024-10-03 00:30:49,719 : INFO: common: yaml file params.yaml loaded successfully]
[2024-10-03 00:30:49,727 : INFO: common: Created directory at : artifacts]
[2024-10-03 00:30:49,734 : INFO: common: Created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-03 00:30:49,739 : INFO: common: Created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
